In [ ]:
from icl.util_classes.arg_classes import NClassificationArgs
from sklearn.metrics import accuracy_score
from icl.utils.load_huggingface_dataset import (
    load_huggingface_dataset_train_and_test,
)
import numpy as np

In [ ]:
def get_acc(y, labels):
    scores = y.predictions[0]
    acc = accuracy_score(labels, np.argmax(scores, axis=1))
    return acc

In [ ]:
all_results = []
for task_name in ["sst2", "trec", "agnews", "emo"]:
    dataset = load_huggingface_dataset_train_and_test(task_name)
    task_results = []
    result_ls = []
    for seed in [42, 43, 44, 45, 46]:
        test_sample = (
            dataset["test"]
            .shuffle(seed=seed)
            .select(range(min(1000, len(dataset["test"]))))
        )
        labels = np.array(test_sample["label"])
        if task_name == "agnews":
            args = NClassificationArgs(
                task_name=task_name, demonstration_shot=3, seeds=[seed]
            )
        else:
            args = NClassificationArgs(
                task_name=task_name, demonstration_shot=5, seeds=[seed]
            )
        results = args.load_result()[0][0]
        (y,) = results

        acc_1 = get_acc(y, labels)
        result_ls.append([acc_1])
    task_results.append(result_ls)
    all_results.append(task_results)